<a href="https://colab.research.google.com/github/bird0401/Instance_level_recognition/blob/main/02_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-lightning==0.7.1

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning import Trainer

import torch,torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.models import resnet18
import torch.utils.data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split

from collections import defaultdict
import os
import random
from tqdm import tqdm
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset

In [ ]:
categories=["car2","uspresident","uspresident_02","tower","tower_02"]
category=categories[0]

In [ ]:
# !cp /content/drive/MyDrive/instance_level_recognition/dog_2.tar.gz .
!cp /content/drive/MyDrive/instance_level_recognition/imgs_{category}.tar.gz .

In [ ]:
!tar -zxvf {category}.tar.gz

In [ ]:
#  !ls images|wc -l

In [ ]:
!rm -rf /content/{category}.tar.gz

In [ ]:
# # 224×224で正規化した画像に変換

class ImageTranform():
  def __init__(self):
    self.data_transform=transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
  def __call__(self,img):
    return self.data_transform(img)

In [ ]:
# transform=ImageTranform()
# img=Image.open("images/Q17504_001.jpg")
# img=transform(img)
# # img=img.numpy().transpose((1,2,0)) # imshow用に次元入れ替え
# img=np.clip(img,0,1)
# plt.imshow(img)
# plt.show()
# print(img.shape)
# print(img.dtype)

In [ ]:
# 実行後のファイルはすでに保存してあるのでコメントアウト

# # image_size=32 #試し用
# image_size=224 

# image_shape=(image_size,image_size,3)

# ファイルの入出力用にglobをインポート
import glob

image_dir = "imgs_"+category
paths=glob.glob(image_dir+"/*")

## 準備

# q_files = defaultdict(list)
imgs,qids=[],[]
transform=ImageTranform()

for path in tqdm(paths):
  try:
    # img=np.array(Image.open(path).convert("RGB").resize((image_size,image_size)))
    img=Image.open(path)
    img=transform(img)
    img=img.numpy()
    img=np.clip(img,0,1)
  except Exception as e:
    print(e)
  else:
    ## キーと画像のリストを保存
    filename=path.split("/")[1]
    qid = filename.split("_")[0][1:]
    qids.append(qid)
    imgs.append(img)
    ## キーから画像を取得できるようにする
    # q_files[qid].append(img)

In [ ]:
# #  pickleで保存
with open('qids_'+category+'.binaryfile','wb') as f:
  pickle.dump(qids,f)

# with open('imgs.binaryfile','wb') as f:
#   pickle.dump(imgs,f)

In [ ]:
# # 保存したファイルをdriveにコピー
!cp /content/{'qids_'+category+'.binaryfile'} /content/drive/MyDrive/instance_level_recognition
# !cp /content/imgs.binaryfile /content/drive/MyDrive/instance_level_recognition

In [ ]:
# 保存したファイルをdriveから取ってくる
!cp /content/drive/MyDrive/instance_level_recognition/{'qids_'+category+'.binaryfile'} .
# !cp /content/drive/MyDrive/instance_level_recognition/imgs.binaryfile .

In [ ]:
# pickleからロード
with open('/content/qids_'+category+'.binaryfile','rb') as f:
  qids=pickle.load(f)
# with open('/content/imgs.binaryfile','rb') as f:
#   imgs=pickle.load(f)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(qids)
y_labels=le.transform(qids)

In [ ]:
n_classes=len(set(y_labels.tolist()))

In [ ]:
imgs=torch.tensor(imgs,dtype=torch.float32)
y_labels=torch.from_numpy(y_labels.astype(np.uint8)).clone()

# X_train, X_valid, X_test = X_train/255.0, X_valid/255.0, X_test/255.0

In [ ]:
#  pickleで保存
with open('content/y_labels_'+category+'.binaryfile','wb') as f:
  pickle.dump(y_labels,f)

with open('content/imgs_'+category+'.binaryfile','wb') as f:
  pickle.dump(imgs,f)

In [ ]:
# 保存したファイルをdriveにコピー
!cp /content/y_labels_tensor.binaryfile /content/drive/MyDrive/instance_level_recognition
!cp /content/imgs_tensor.binaryfile /content/drive/MyDrive/instance_level_recognition

In [ ]:
# 保存したファイルをdriveから取ってくる
!cp /content/drive/MyDrive/instance_level_recognition/{'y_labels_'+category+'.binaryfile'} .
!cp /content/drive/MyDrive/instance_level_recognition/'imgs_'+category+'.binaryfile' .

In [ ]:
# pickleからロード
with open('content/y_labels_'+category+'.binaryfile','rb') as f:
  y_labels=pickle.load(f)
with open('content/imgs_'+category+'.binaryfile','rb') as f:
  imgs=pickle.load(f)

In [ ]:
# 統合
dataset=torch.utils.data.TensorDataset(imgs,y_labels)
dataset

In [ ]:
n_train = int(len(dataset) * 0.6)
n_val = int((len(dataset) - n_train) * 0.5)
n_test = len(dataset) - n_train - n_val
n_train, n_val, n_test

In [ ]:
torch.manual_seed(0)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])
# len(X_train),len(X_valid),len(X_test)

# Resnet18

In [ ]:
class TrainNet(pl.LightningModule):
  def train_dataloader(self):
    return torch.utils.data.DataLoader(train,self.batch_size,shuffle=True)
  def training_step(self,batch,batch_nb):
    x,t=batch
    y=self.forward(x)
    loss=self.lossfun(y,t)
    results={"loss":loss}
    return results

class ValidationNet(pl.LightningModule):
  def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)
  def validation_step(self,batch,batch_nb):
    x,t=batch
    y=self.forward(x)
    loss=self.lossfun(y,t)
    y_label=torch.argmax(y,dim=1)
    acc=torch.sum(t==y_label)/len(t)
    results={"val_loss":loss,"val_acc":acc}
    return results
  def validation_end(self,outputs):
    avg_loss=torch.stack([x["val_loss"] for x in outputs]).mean()
    avg_acc=torch.stack([x["val_acc"] for x in outputs]).mean()
    results={"val_loss":avg_loss,"val_acc":avg_acc}
    return results

class TestNet(pl.LightningModule):
    def test_dataloader(self):
        return torch.utils.data.DataLoader(test, self.batch_size)

    def test_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'test_loss': loss, 'test_acc': acc}
        return results

    def test_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        results = {'test_loss': avg_loss, 'test_acc': avg_acc}
        return results

In [ ]:
resnet=resnet18(pretrained=True)

class Net(TrainNet,ValidationNet, TestNet):
  def __init__(self,batch_size=256):
    super().__init__()
    self.batch_size=batch_size
    self.conv=resnet
    self.fc=nn.Linear(1000,n_classes)
    for param in self.conv.parameters():
      param.requires_grad=False
      
  def lossfun(self, y, t):
      return F.cross_entropy(y, t)

  def configure_optimizers(self):
      return torch.optim.SGD(self.parameters(), lr=0.01)

  def forward(self, x):
      x = self.conv(x)
      x = self.fc(x)
      return x

In [ ]:
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False
torch.manual_seed(0)

net=Net()
trainer=Trainer(gpus=1,max_epochs=10,batch_size=1024)
trainer.fit(net)

Validation sanity check:   0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Validating:   0%|          | 0/2 [00:00<?, ?it/s]

Validating:   0%|          | 0/2 [00:00<?, ?it/s]

Validating:   0%|          | 0/2 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/2 [00:00<?, ?it/s]

Validating:   0%|          | 0/2 [00:00<?, ?it/s]

Validating:   0%|          | 0/2 [00:00<?, ?it/s]

Validating:   0%|          | 0/2 [00:00<?, ?it/s]

1

# Results

In [ ]:
trainer.test()
trainer.callback_metrics

Testing:   0%|          | 0/2 [00:00<?, ?it/s]

----------------------------------------------------------------------------------------------------
TEST RESULTS
{}
----------------------------------------------------------------------------------------------------


{'epoch': 9,
 'loss': 1.032576560974121,
 'test_acc': 0.4496504068374634,
 'test_loss': 1.9942290782928467,
 'val_acc': 0.4688471257686615,
 'val_loss': 1.9994654655456543}